In [ ]:
#!pip install selenium

In [ ]:
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time
import json
import re
import pickle

In [ ]:
#List of CNPJs to be processed
cnpjs=\
"""
37.093.442/0001-30
37.092.411/0001-64
27.249.022/0001-46
36.400.027/0001-19
37.074.530/0001-94"""

cnpjs = cnpjs.split()

In [ ]:
# Criação da função de consulta

def consulta_patrimonio(CNPJ):
    #Definição dos objetos para uso da consulta com o navegador chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    #chrome_driver_path = '/home/lucas/my_project_dir/chromedriver'
    chrome_driver_path ="C:\selenium\chromedriver_win32\chromedriver.exe"
    service = Service(executable_path=chrome_driver_path)

    #Definindo a URL com base na função de inspeção do hmtl e o CNPJ requisitado na função
    #do navegador para a página de consulta dos CNPJs
    url= "https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CConsolFdo/ResultBuscaParticFdo.aspx?CNPJNome="+CNPJ+"&TpPartic=0&Adm=false&SemFrame=.aspx"
        
    #Acessando a URL com o driver
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    
    #Selecionando o botão de pesquisa do CNPJ incluído na url anterior
    submit_button = driver.find_element(By.ID, "ddlFundos__ctl0_Linkbutton2")
    submit_button.click()

    
    time.sleep(2)

    #Dentro da página procuramos pelo botão que direcionará para a página dos balanços
    submit_button = driver.find_element(By.ID, "Hyperlink2")
    submit_button.click()

    time.sleep(2)
    #Gravação a url da página obtida após a seleção do botão de balanços na variável "page_3"
    page_3 = driver.page_source
    competencia = driver.find_element(By.ID,'ddComptc')
    
    for i in competencia.find_elements(By.TAG_NAME,'option'):
        if i.get_attribute('selected'):
            data_atual = i.get_attribute('value')
    
    time.sleep(2)
    #finalizei a consulta com o scraper pois já nos encontramos na página necessária para 
    # obtenção dos dados
    driver.quit()

    #Utilização do método "read_hmtl" para leitura das tabela da url do objetvo "page_3"
    tabela = pd.read_html(page_3, header=0)

    #A variável acima retorna duas tabelas, sendo que as informação que queremos está na segunda
    #tabela, no caso de índice 1
    tabela_2 = tabela[1]

    #Manipulações necessários para tratamento da tabela e seleção das variáveis de interesse
    #após alguns teste, demonstrou-se necessário incluir um "try/except" pois em alguns cenários
    #o retorno do nome das colunas não vinham com espaço entre algumas palavras
    try:
        tabela_3 = tabela_2[['Dia','Patrimônio Líquido (R$)','Data da próxima informação do PL']]
        tabela_4 = tabela_3.fillna(value = 0, method=None)

        valor_fundo = tabela_4['Patrimônio Líquido (R$)'].to_list()
        
    #Nesse except não há espaço entre a palavra "Líquido e (R$) como também "próxima" e "informação"
    except:
        tabela_3 = tabela_2[['Dia','Patrimônio Líquido(R$)','Data da próximainformação do PL']]
        tabela_4 = tabela_3.fillna(value = 0, method=None)

        valor_fundo = tabela_4['Patrimônio Líquido(R$)'].to_list()
    
        
    #Após criação das listas acima, procuramos o índice com maior valor da lista para o a lista de 
    #valor do fundo para obtermos a data, na lista "dias" e "date" a partir desse índice
    index_max=[]
    for dado in valor_fundo:
        if dado != 0:
            valor_fundo.index(dado)
            index_max.append(valor_fundo.index(dado))

    dia_max = index_max[-1] + 1

    #determinação da última data a partir da variável "dia_max" com o mes_ano acima
    date_2 = f"{str(dia_max)}/"+ data_atual

    #Criação de um dicionário para armazenamento da consulta
    consulta_dict = {"CNPJ":CNPJ,"Data":date_2,"Patrimonio":valor_fundo[index_max[-1]]}

        

    return consulta_dict

In [ ]:
#Criação de lista para armazenamento do resultado do loop de consulta
consulta_retorno = []

for CNPJ in cnpjs:
    resultado = consulta_patrimonio(CNPJ)
    consulta_retorno.append(resultado)
    
#inclusão de uma pausa no código para evitar o bloquei do site
    time.sleep(2)

with open('consulta_retorno.json', 'w') as f:
    json.dump(consulta_retorno, f)

In [ ]:
def composicao_carteira(CNPJ):
    #Definição dos objetos para uso da consulta com o navegador chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    #chrome_driver_path = '/home/lucas/my_project_dir/chromedriver'
    chrome_driver_path ="C:\selenium\chromedriver_win32\chromedriver.exe"
    service = Service(executable_path=chrome_driver_path)

    #Definindo a URL com base na função de inspeção do hmtl e o CNPJ requisitado na função
    #do navegador para a página de consulta dos CNPJs
    url= "https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CConsolFdo/ResultBuscaParticFdo.aspx?CNPJNome="+CNPJ+"&TpPartic=0&Adm=false&SemFrame=.aspx"

    #Acessando a URL com o driver
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)

    #Selecionando o botão de pesquisa do CNPJ incluído na url anterior
    submit_button = driver.find_element(By.ID, "ddlFundos__ctl0_Linkbutton2")
    submit_button.click()
    time.sleep(3)

    #Dentro da página procuramos pelo botão que direcionará para a página dos balanços
    submit_button = driver.find_elements(By.ID,"Hyperlink1")
    submit_button[1].click()
    time.sleep(3)

    #Gravação a url da página obtida após a seleção do botão de balanços na variável "page_3"
    page_3 = driver.page_source

    #finalizei a consulta com o scraper pois já nos encontramos na página necessária para 
    #obtenção dos dados
    driver.quit()

    #Utilização do método "read_hmtl" para leitura das tabela da url do objetvo "page_3"
    #Definição da tabela de índice 01 para sempre pegar a carteira completamente disponível
    tabela_carteira = pd.read_html(page_3, header=3)
    df1 = pd.DataFrame(tabela_carteira[1])
    df1['CNPJ'] = CNPJ
        
    return df1

In [ ]:
carteira_fundos = pd.DataFrame()
for cnpj in cnpjs:
    print(f'CNPJ {cnpj} adicionado à tabela \"carteira_fundos\"')
    carteira_fundos = pd.concat([carteira_fundos,composicao_carteira(cnpj)], ignore_index = True)
    time.sleep(5)
    
#Armazenamento do dataframe em um arquivo serializado a partir do DataFrame do Pandas
carteira_fundos.to_pickle('carteira_fundos.pkl')
df_pickle = pd.read_pickle('carteira_fundos.pkl')
df_pickle.head(10)